# t-SNE Analysis of Deepfake Detection Features

This notebook demonstrates how t-SNE is applied to visualize high-dimensional feature embeddings extracted from deepfake detection models. The purpose is to analyze clustering behavior between real and fake samples before and after model tuning.

This version is curated for external readers and reviewers (e.g., admissions committees) to showcase data analysis, function design, and visualization skills.

## Step 1: Load and Prepare Data
We begin by loading preprocessed features and labels extracted from our image classification model.

In [ ]:
from datasets import load_dataset
import os
from datasets import concatenate_datasets
from transformers import AutoModelForImageClassification, AutoModel, TrainingArguments, Trainer
from transformers import AutoImageProcessor
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

import numpy as np
import torch
import evaluate
import torch.nn.functional as F
from scipy.special import softmax

#----img prepro func-----
from torchvision.transforms import (
    RandomCrop,
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    RandomApply,
    RandomChoice,
    RandomRotation,
    Resize,
    RandomErasing,
    ToTensor,
    ColorJitter,
    ToPILImage
)

# Test Result

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
labels = ["REAL", "FAKE"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
    
def reformat_dataset(example):
    example["label"] = label2id[example["json"]["label"].upper()]
    return example

#-----load dataset----
WEBDATASET_ROOT = "data/webdataset/"
fnl = os.listdir(WEBDATASET_ROOT)
fnl = [f for f in fnl if 'ipynb_checkpoints' not in f]

test_data_l = {}
for dataset in fnl:
    test_data = load_dataset("webdataset", data_dir=WEBDATASET_ROOT+dataset, split="test")
    print(dataset, len(test_data))
    
    test_data = test_data.map(reformat_dataset, num_proc=os.cpu_count())
    if dataset in ['ucf_selfie_dataset','covid_fmd_dataset','FMD_mask_data','mask_detection_data']:
        test_data = test_data.rename_column("jpg", "image")
    elif dataset == 'testing':
        test_data = test_data.rename_column("jpeg", "image")
    else:
        test_data = test_data.rename_column("png", "image")
    test_data = test_data.remove_columns(["json"])
    test_data_l[dataset] = test_data

In [ ]:
ds_test_real_l = []
ds_test_fake_l = []
for name, ds in test_data_l.items():
    ds_real = ds.filter(lambda x:x["label"]==0, num_proc=os.cpu_count()) 
    if name in dataset_n_lst:
        ds_fake = ds.filter(lambda x:x["label"]==1, num_proc=os.cpu_count())
        
    if any(ds_real):
        ds_test_real_l.append(ds_real)
    if any(ds_fake):
        ds_test_fake_l.append(ds_fake)

    print(f'name={name}, tot={len(ds)}, real={len(ds_real)}, fake={len(ds_fake)}')

In [ ]:
def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

test_ds.set_transform(preprocess_val)

## Load Model

In [ ]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

print(size, crop_size)

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [ ]:
saved_image_processor = AutoImageProcessor.from_pretrained(checkpoint_path)
saved_model = AutoModelForImageClassification.from_pretrained(
    checkpoint_path,
    label2id=label2id,
    id2label=id2label,
)

device = torch.device("cuda")
saved_model = saved_model.to(device)

## Design Metrics

In [ ]:
def evaluate_all(df):
    roc_auc_metric = evaluate.load("roc_auc")
    acc_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    
    preds = df["pred"].values
    pred_probs = df["pred_prob"].values
    refs = df["label"].values

    acc = acc_metric.compute(predictions=preds, references=refs)["accuracy"]
    roc_auc = roc_auc_metric.compute(prediction_scores=pred_probs, references=refs)["roc_auc"]
    precision= precision_metric.compute(predictions=preds, references=refs, average=None)
    recall = recall_metric.compute(predictions=preds, references=refs, average=None)
    f1 = f1_metric.compute(predictions=preds, references=refs, average=None)

    result = {"accuracy": acc,
              "AUC": roc_auc,
              "precision": precision,
              "recall": recall,
              "f1": f1}
    return result

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
from tqdm import tqdm

def get_inference_result(model, test_data_loader):
    
    model.eval()
    pred_list = []
    pred_prob_list = []


    for step, batch in enumerate(tqdm(test_data_loader)):

        batch = {"pixel_values": batch["pixel_values"].to(device),
                "labels": batch["labels"].to(device)}
        with torch.no_grad():
            outputs = saved_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        pred_probs = outputs.logits.softmax(dim=-1)[:,1]
        references = batch["labels"]
        pred_list += list(predictions)
        pred_prob_list+= list(pred_probs)

    return (pred_list, pred_prob_list)

def format_evaluate_result_df(result):
    new_dic = {}
    new_dic["Accuracy"] = round(result["accuracy"],4)
    
    if "AUC" in result:
        new_dic["AUC"] = round(result["AUC"],4)
    new_dic["Precision 0"] = round(result["precision"]["precision"][0], 4)
    new_dic["Precision 1"] = round(result["precision"]["precision"][1], 4)
    new_dic["Recall 0"] = round(result["recall"]["recall"][0], 4)
    new_dic["Recall 1"] = round(result["recall"]["recall"][1], 4)
    new_dic["F1 0"] = round(result["f1"]["f1"][0], 4)
    new_dic["F1 1"] = round(result["f1"]["f1"][1], 4)
    
    result_df = pd.DataFrame([new_dic])
    return result_df

## Inference Model

In [ ]:
from torch.utils.data import DataLoader

# Evaluation dataloader
eval_batch_size = 1024
test_dataloader = DataLoader(test_ds, collate_fn=collate_fn, batch_size=eval_batch_size, pin_memory=True, num_workers=8)

In [ ]:
inf_result = get_inference_result(saved_model, test_dataloader)

In [ ]:
saved_model_embedding = AutoModel.from_pretrained(
    checkpoint_path,
    label2id=label2id,
    id2label=id2label,
)

device = torch.device("cuda")
saved_model_embedding = saved_model_embedding.to(device)

def detect_deepfake_embeddings(model, test_data_loader):
        
    model.eval()
    pred_embedding_list = []

    for step, batch in enumerate(tqdm(test_data_loader)):
        # print(batch)
        # print(step)
        batch = {"pixel_values": batch["pixel_values"].to(device),
                "labels": batch["labels"].to(device)}
        batch.pop('labels')
        with torch.no_grad():
            last_hidden_state = model(**batch).last_hidden_state
            # hidden_output = torch.cat(last_hidden_state, dim=0)
        split_tensors = torch.split(last_hidden_state, 1, dim=0)
        pred_embedding_list += list(split_tensors)
        
    return pred_embedding_list

In [ ]:
inf_embedding = detect_deepfake_embeddings(saved_model_embedding, test_dataloader)

In [ ]:
inf_embedding = [tensor.cpu().numpy() for tensor in inf_embedding]
test_embedding = {"key": test_df['key'], "embeddings": list(inf_embedding)}
test_embedding_df = pd.DataFrame(test_embedding)
test_df = test_df.merge(test_embedding_df, how='left', on='key')
test_df.head()

## Design Visualization Function

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import hdbscan
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

In [ ]:
def get_flatten_data(df, dataset_list, target_column, group, error=False, error_ind=None):
    """Flatten data into 1-dimensional data"""
    df = df[df['dataset'].isin(dataset_list)]
    processed_data = df[target_column].apply(lambda x: x.reshape(-1)).to_numpy()
    processed_data = np.vstack(processed_data).astype(np.float32)
    labels = df['label'].to_list()
    sources = df[group].to_list()
        
    return processed_data, labels, sources


def apply_dimensionality_reduction(data, method='tsne', n_components=2, **kwargs):
    """
    :param data: numpy array, shape (n_samples, n_features)
    :param method: 'tsne' or 'pca'
    :param n_components: number of dimensions to reduce to
    :return: numpy array, shape (n_samples, n_components)
    """
    if method == 'tsne':
        return TSNE(n_components=n_components, random_state=42, **kwargs).fit_transform(data)
    elif method == 'pca':
        return PCA(n_components=n_components, random_state=42, **kwargs).fit_transform(data)
    else:
        raise ValueError("Method must be 'tsne' or 'pca'")
      
    
def visualize_plotly_custom(X, labels, sources, title):
    
    df = pd.DataFrame({
        'x': X[:, 0],
        'y': X[:, 1],
        'label': labels,
        'source': sources
    })
    df['label'] = df['label'].replace({0: 'real', 1: 'fake'})

    color_map = {
        'fake': 'rgba(244, 97, 151, 0.5)',
        'real': 'rgba(65, 157, 120, 0.5)'
    }

    marker_map = {
        source: marker for source, marker in zip(
            df['source'].unique(),
            ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'triangle-left', 'triangle-right', 'pentagon', 
             'hexagram', 'star', 'bowtie', 'star-diamond', 'diamond-tall', 'diamond-wide', 'hourglass',
             'circle-open', 'square-open', 'diamond-open', 'cross-open', 'x-open', 
             'triangle-up-open', 'triangle-down-open', 'triangle-left-open', 'triangle-right-open', 'pentagon-open', 
             'hexagram-open', 'star-open', 'bowtie-open', 'star-diamond-open', 'diamond-tall-open', 
             'diamond-wide-open', 'hourglass-open']
        )
    }

    fig = go.Figure()

    for label in ['fake', 'real']:
        for source in list(set(sources)):
            subset = df[(df['label'] == label) & (df['source'] == source)]
            fig.add_trace(go.Scattergl(
                x=subset['x'],
                y=subset['y'],
                mode='markers',
                marker=dict(
                    color=color_map[label],
                    size=10,
                    symbol=marker_map[source],
                    # line=dict(width=0.3, color='DarkSlateGrey')
                ),
                name=f'{label}-{source}',
                hovertext=f'{label}-{source}',
                hoverinfo='text'
            ))
            

    fig.update_layout(
        title=title,
        xaxis_title='t-SNE 1',
        yaxis_title='t-SNE 2',
        legend_title='Label-Source',
        width=1000,
        height=800,
        plot_bgcolor='rgb(240,240,240)',
        hovermode='closest'
    )

    fig.write_html(f"visualization/{title}.html")
    
    fig.show()

## Step 2: t-SNE Visualization - Before Tuning
We apply t-SNE to visualize the embeddings of the model before tuning. This helps identify overlapping areas between real and fake samples.

In [ ]:
dataset_list = test_df['dataset'].unique().tolist()
target_column = 'embeddings'
group = 'dataset'
reduced_method = 'tsne'
detail_info = 'v4_model_testing_dataset'

raw_processed_data, raw_labels, raw_sources = get_flatten_data(test_df, dataset_list, target_column, group)
raw_reduced_data = apply_dimensionality_reduction(raw_processed_data, reduced_method)
visualize_plotly_custom(raw_reduced_data, raw_labels, raw_sources, f'{reduced_method} visualization of {detail_info} focusing on raw signals')

## Step 3: t-SNE Visualization - After Tuning
After threshold optimization and fine-tuning, the separation between clusters becomes clearer. This step visualizes the improvement.

In [ ]:
dataset_list = test_df['dataset'].unique().tolist()
target_column = 'embeddings'
group = 'dataset'
reduced_method = 'tsne'
detail_info = 'v5_model_testing_dataset'

raw_processed_data, raw_labels, raw_sources = get_flatten_data(test_df, dataset_list, target_column, group)
raw_reduced_data = apply_dimensionality_reduction(raw_processed_data, reduced_method)
visualize_plotly_custom(raw_reduced_data, raw_labels, raw_sources, f'{reduced_method} visualization of {detail_info} focusing on raw signals')

## Summary
t-SNE analysis enabled us to visually diagnose model misclassifications and improve overall performance by fine-tuning thresholds and separating ambiguous samples.